In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "CBIR"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.extractNeuralFeatures import extract_features

In [4]:
chosen_net = networks.ALEXNET
linear_layers_to_remove = 3
X_train_t, X_train_n, y_train, X_test_t, X_test_n, y_test, fine_tune_layers, _ = extract_features(train_set=datasets.TRAINING_LABELED,
                                                                                            test_set=datasets.TEST,
                                                                                            network=chosen_net,
                                                                                            layers_to_remove=linear_layers_to_remove, 
                                                                                            cuda=cuda)
#print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
#print("---------------------------------------------------------------------------------")
#print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

Found an existing set of features in: ./Storage/neural_features/Train_AlexNet_minus3_train_small.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Train: 5020
Feature vector shape of Train: (5020, 9216)
Label vector shape of Train: (5020,)
---------------------------------------------------------------------------------
Found an existing set of features in: ./Storage/neural_features/Test_AlexNet_minus3_val_set.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Test: 11994
Feature vector shape of Test: (11994, 9216)
Label vector shape of Test: (11994,)
---------------------------------------------------------------------------------


In [5]:
chosen_net = networks.ALEXNET
layers_to_remove = 3

#X_train_ale_t, X_train_ale_n, y_train, X_test_ale_t, X_test_ale_n, y_test, fine_tune_layers = extract_features(train_set=datasets.TRAINING_LABELED,
X_unlabled_t, X_unlabled_n, _, X_test_t, X_test_n, y_test, fine_tune_layers, _ = extract_features(train_set=datasets.TRAINING_UNLABELED,
                                                                                            test_set=datasets.TEST,
                                                                                            network=chosen_net,
                                                                                            layers_to_remove=layers_to_remove,
                                                                                            cuda=cuda)
#print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
#print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

Found an existing set of features in: ./Storage/neural_features/Train_AlexNet_minus3_train_unlabeled.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Train: 113455
Feature vector shape of Train: (113455, 9216)
Label vector shape of Train: (113455,)
---------------------------------------------------------------------------------
Found an existing set of features in: ./Storage/neural_features/Test_AlexNet_minus3_val_set.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Test: 11994
Feature vector shape of Test: (11994, 9216)
Label vector shape of Test: (11994,)
---------------------------------------------------------------------------------


In [6]:
import csv
translate_label = []

# Read the CSV file
with open('disambiguation.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        label = row[1]
        # Ensure the list is large enough to hold the current index
        
        translate_label.append(label)

print(translate_label)


['macaron', 'beignet', 'cruller', 'cockle_food', 'samosa', 'tiramisu', 'tostada', 'moussaka', 'dumpling', 'sashimi', 'knish', 'croquette', 'couscous', 'porridge', 'stuffed_cabbage', 'seaweed_salad', 'chow_mein', 'rigatoni', 'beef_tartare', 'cannoli', 'foie_gras', 'cupcake', 'osso_buco', 'pad_thai', 'poutine', 'ramen', 'pulled_pork_sandwich', 'bibimbap', 'chicken_kiev', 'apple_pie', 'risotto', 'fruitcake', 'chop_suey', 'haggis', 'scrambled_eggs', 'frittata', 'scampi', 'sushi', 'orzo', 'fritter', 'nacho', 'beef_stroganoff', 'beef_wellington', 'spring_roll', 'savarin', 'crayfish_food', 'souffle', 'adobo', 'streusel', 'deviled_egg', 'escargot', 'club_sandwich', 'carrot_cake', 'falafel', 'farfalle', 'terrine', 'poached_egg', 'gnocchi', 'bubble_and_squeak', 'egg_roll', 'caprese_salad', 'sauerkraut', 'creme_brulee', 'pavlova', 'fondue', 'scallop', 'jambalaya', 'tempura', 'chocolate_cake', 'potpie', 'spaghetti_bolognese', 'sukiyaki', 'applesauce', 'baklava', 'salisbury_steak', 'linguine', 'eda

In [7]:
#normalizzazione features:

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler



# Min-Max Scaling
min_max_scaler = MinMaxScaler()
normalized_features = min_max_scaler.fit_transform(X_train_n)

# Standardization
standard_scaler = StandardScaler()
X_train_n = standard_scaler.fit_transform(normalized_features)


In [8]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

if cuda:
    #boh
    pass
else:    
    X_train, X_val, y_train, y_val = train_test_split(X_train_n, y_train, test_size=0.1, random_state=42)

In [17]:
from sklearn.metrics.pairwise import pairwise_distances
d = pairwise_distances(X_train, X_val, metric='sqeuclidean', n_jobs=-1)#1o argument->le query, 2o->le pic da "indicizazre"

In [18]:
print(d.shape)
print(d.dtype)

(4518, 502)
float32


In [19]:

def sort_indices_by_distance(matrix):
    sorted_indices = [np.argsort(row).tolist() for row in matrix]
    return sorted_indices


def threshold_matrix(matrix, threshold):  #ha senso? se sono valori grandi sono in fondo al ranking tanto..
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j]>=threshold:
                matrix[i][j]=np.iinfo(matrix.dtype).max
    return matrix

#per esempio:
D = np.array([
    [0, 2, 3, 4],
    [2, 0, 1, 5],
    [3, 1, 0, 6],
    [4, 5, 6, 0]
])
#print(threshold_matrix(D, 5))
#sorted_indices = sort_indices_by_distance(D)#[[0, 1, 2, 3], [1, 2, 0, 3], [2, 1, 0, 3], [3, 0, 1, 2]]

sorted_indices = sort_indices_by_distance(d)



In [20]:
# who are the relevant images of the columns (i.e. the second argument in pairwise_dist) wrt a "query" (an image, i.e. row in the Distances matrix)?
#guardo la label della pic che corrisponde alla riga ->le pic di colonna sono rilevanti wrt la pic di riga sse hanno la stessa label
#devo cotruire matrice delle rilevanze
qrels=[]
for i in range(len(y_train)):

    labelOfTheRowPic = y_train[i]
    newRow=[]
    for j in range(len(y_val)):
        if labelOfTheRowPic==y_val[j]:
            newRow.append(1)
        else:
            newRow.append(0)
    qrels.append(newRow)
print(qrels[:10])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
len(qrels)

4518

In [23]:
#compute avarage precision@K:  (# of recommended items @k that are relevant) / (# of recommended items @k ->k) ... all of this divided by the number of queries
k=5
precisions=[]
for i in range(len(qrels)):
    relevantPicsNailed=0
    for j in range(k):
        if qrels[i][sorted_indices[i][j]]==1:
            relevantPicsNailed+=1
    precisions.append(relevantPicsNailed/k)
print("avarage precision at k = ", np.mean(precisions))

avarage precision at k =  0.008942009738822488


con pairwise_distances(X_train, X_val, metric='sqeuclidean' ..) e features grandi 9216 di alexnet

AP@5 = 0.0122, con in totale 276 pic rilevanti retrieved prima della k-esima posizione dei risultati

AP@10 =0.0101 con in totale 460 pic rilevanti retrieved prima della k-esima posizione dei risultati
...se metto dist euclidea ho gli stessi risultati di sopra.

con citiblock: 
AP@10 =0.009
AP@5 = 0.0108

NORMALIZZANDO LE FEATURES:
con citiblock: 
AP@10 =0.0078
AP@5 =0.0088
con sqeuclidean:
AP@10: 0.0078
AP@5: 0.0089